Neo4j Environment Setup

In [ ]:
# neo4j env. setup
from langchain_community.graphs import Neo4jGraph

url = "bolt://34.205.41.21:7687"
username = "neo4j"
password = "trades-window-pilots"

graph = Neo4jGraph(
    url = url,
    username = username,
    password = password
)

In [ ]:
# pip install --upgrade --quiet  langchain
# pip install --upgrade --quiet  langchain-openai

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

import os
os.environ["OPENAI_API_KEY"] = "API_KEY"


NEO4J_URI = "bolt://34.205.41.21:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "trades-window-pilots"

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)


chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

chain.run("What is Walter Elias Disney's birthdate?")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Walter Elias Disney"}) 
RETURN p.birthdate
Full Context:
[{'p.birthdate': 'December 5, 1901'}]

> Finished chain.


"Walter Elias Disney's birthdate is December 5, 1901."

In [3]:
chain.invoke("Who is sibling of Walt Disney?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person {name: "Walt Disney"})-[:SIBLING]->(p2:Person)
RETURN p2.name
Full Context:
[]

> Finished chain.


{'query': 'Who is sibling of Walt Disney?',
 'result': 'Walt Disney had a brother named Roy O. Disney.'}

In [4]:
chain.invoke("Who is sibling of Walt Disney? Do not make up any information.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person {name: "Walt Disney"})-[:SIBLING]->(sibling)
RETURN sibling.name
Full Context:
[]

> Finished chain.


{'query': 'Who is sibling of Walt Disney? Do not make up any information.',
 'result': "I don't know the answer to that question."}

In [ ]:
QUERY = "MATCH (n) RETURN n"

graph.query(QUERY)

Information Extraction Pipeline (Extracting structured info. with OpenAI functions using LangChain)

In [ ]:
# import

from langchain_community.graphs.graph_document import(
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

In [ ]:
# initialize classes for property, node, relationship, KG links

class Property(BaseModel):
    """A single property consisting of key and value"""
    key: str = Field(..., description="key")
    value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(None, description="List of relationship properties")

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(..., description="List of relationships in the knowledge graph")

In [ ]:
# details functions

def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)
##### Q: why first word is lower & latter words are capitalized? What is the output? #####

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
        return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties
    
def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id = node.id.title(), type = node.type.capitalize(), properties=properties
    )

def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

In [ ]:
import os
from langchain.chains.openai_functions import(
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
os.environ["OPENAI_API_KEY"] = "API_KEY"
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature=0)

In [ ]:
def get_extraction_chain(
        allowed_nodes: Optional[List[str]] = None,
        allowed_rels: Optional[List[str]] = None):
    prompt = ChatPromptTemplate.from_messages(
        [(
            "system",
            f"""# Knowledge Graph Instructions for GPT-4
            ## 1. Overview
            You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
            - **Nodes** represent entities and concepts. They're similar to Wikipedia nodes. 
            - The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
            ## 2. Labeling Nodes
            - **Consistency**: Ensure you use basic or elementary types for node labels.
                - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
            - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
            {'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
            {'- **Allowed Relationship Types:**' + ", ".join(allowed_rels) if allowed_rels else ""}
            ## 3. Handling Numerical Data and Dates
            - Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
            - **No Separate Nodes for Dates/Numbers**: Do not create seperate nodes for dates or numerical values. Always attach them as attributes or properties of nodes. 
            - **Property Format**: Properties must be in a key-value format.
            - **Quotation Marks**: Never use escaped single or double quotes within property values.
            - **Naming Convention**: Use camelCase for property keys, e.g., 'birthDate'.
            ## 4. Coreference Resolution
            - **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency. 
            If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
            always use the most complete identifier for that entity throughtout the knowledge graph. In this example, use "John Doe" as the entity ID.
            Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
            ## 5. Strict Compliance
            Adhere to the rules strictly. Non-compliance will result in termination.
        """),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose = False)

In [ ]:
def extract_and_store_graph(
        document: Document,
        nodes: Optional[List[str]] = None,
        rels: Optional[List[str]] = None) -> None:
    
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke(document.page_content)['function']

    # Construct a graph document
    graph_document = GraphDocument(
        nodes = [map_to_base_node(node) for node in data.nodes],
        relationships= [map_to_base_relationship(rel) for rel in data.rels],
        source = document
    )

    # store information into a graph
    graph.add_graph_documents([graph_document])

Evaluation

In [ ]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter

In [ ]:
# Read the wikipedia article
raw_documents = WikipediaLoader(query = "Walt Disney").load()

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size = 2048, chunk_overlap = 24)

# only take the first the raw documents
documents = text_splitter.split_documents(raw_documents[:3])

# 54.5s

In [ ]:
# from tqdm import tqdm

# for i, d in tqdm(enumerate(documents), total = len(documents)):
#     extract_and_store_graph(d)

In [ ]:
# Delete the graph
# graph.query("MATCH (n) DETACH DELETE n")

In [ ]:
from tqdm import tqdm

# Specify which node labels should be extracted by the LLM
allowed_nodes = ["Person", "Company", "Location", "Event", "Movie", "Service", "Award"]

for i, d in tqdm(enumerate(documents), total = len(documents)):
    extract_and_store_graph(d, allowed_nodes)

Entity Disambiguation (Skipped)

RAG Application

In [ ]:
# Query the knowledge graph in a RAG Application

from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    graph = graph,
    cypher_llm = ChatOpenAI(temperature=0, model = "gpt-3.5-turbo-0125"),
    qa_llm= ChatOpenAI(temperature=0, model = "gpt-3.5-turbo-0125"),
    validate_cypher = True, # validate relationship directions
    verbose = True
)

In [ ]:
cypher_chain.invoke({"query": "What are the movies produced by Walter Disney? List 3. "})

In [ ]:
cypher_chain.invoke({"query": "What are the movies produced by Walter Elias Disney?"})

In [ ]:
cypher_chain.invoke("What is sibling of Walt Disney? Only answer if the answer is found in the graph database.")

In [ ]:
graph